<a href="https://colab.research.google.com/github/0Sa-ad0/Photo-Resizer/blob/main/Photo_Resizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install Pillow

In [3]:
from PIL import Image
import os

def resize_image_for_embassy(input_path, output_path):
    """
    Resize image to 400x514 pixels (45mmx35mm) with file size < 150KB
    Maintains quality while ensuring embassy requirements are met
    """
    # Open the image
    img = Image.open(input_path)

    # Convert to RGB if necessary (handles PNG with transparency, etc.)
    if img.mode in ('RGBA', 'LA', 'P'):
        background = Image.new('RGB', img.size, (255, 255, 255))
        if img.mode == 'P':
            img = img.convert('RGBA')
        background.paste(img, mask=img.split()[-1] if img.mode in ('RGBA', 'LA') else None)
        img = background

    # Resize to exact dimensions required
    target_size = (400, 514)
    img_resized = img.resize(target_size, Image.Resampling.LANCZOS)

    max_file_size = 150 * 1024  # 150KB in bytes

    # Determine output format from extension
    output_ext = os.path.splitext(output_path)[1].lower()

    if output_ext == '.png':
        # For PNG, try different compression levels
        for compress_level in range(6, 10):  # 6 is good balance, 9 is max compression
            img_resized.save(output_path, 'PNG', optimize=True, compress_level=compress_level)
            file_size = os.path.getsize(output_path)

            if file_size <= max_file_size:
                print(f"✓ Image saved successfully!")
                print(f"  Dimensions: {img_resized.size}")
                print(f"  File size: {file_size / 1024:.2f} KB")
                print(f"  Format: PNG (compress_level={compress_level})")
                return True

        # If PNG still too large, convert to JPEG
        print("PNG file too large, converting to JPEG...")
        output_path = output_path.replace('.png', '.jpg')

    # Use JPEG format with quality adjustment
    quality = 95
    while quality > 10:
        img_resized.save(output_path, 'JPEG', quality=quality, optimize=True)
        file_size = os.path.getsize(output_path)

        if file_size <= max_file_size:
            print(f"✓ Image saved successfully!")
            print(f"  Dimensions: {img_resized.size}")
            print(f"  File size: {file_size / 1024:.2f} KB")
            print(f"  Quality: {quality}")
            print(f"  Format: JPEG")
            return True

        quality -= 5

    print("Warning: Could not reduce file size below 150KB while maintaining reasonable quality")
    return False

# Usage
input_image = "SAAD.png"
output_image = "SAAD-SG.jpg"  # Changed to .jpg for better compression

resize_image_for_embassy(input_image, output_image)

✓ Image saved successfully!
  Dimensions: (400, 514)
  File size: 44.80 KB
  Quality: 95
  Format: JPEG


True